In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive

/content/drive


In [3]:
%cd Shareddrives/blisssinghal-dataset/

/content/drive/Shareddrives/blisssinghal-dataset


In [4]:
%pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.3 MB/s eta 0:00:00


In [5]:
import glob
import numpy as np
import pandas as pd
import mne
import os
import matplotlib.pyplot as plt
import sklearn 
import keras
import pickle
import pickletools
import sklearn.neighbors
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler

In [6]:
from keras.layers import (LSTM, Dense, RNN, Flatten, Dropout, SimpleRNN, Conv1D)
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers


In [7]:
def flatten(names):
    new_array = []
    for i in range(len(names)):
        for j in range(len(names[i])):
            new_array.append(names[i][j])
    return new_array

In [8]:
def save_epochs(epochs, file):
    with open(file, 'wb') as file: 
        pickle.dump(epochs, file)

In [9]:
def load_epochs(file_name):
    with open(file_name, 'rb') as f:
        epochs = pickle.load(f)
    return epochs

In [10]:
epochs_array = load_epochs("epoch_files/epochs_with_ica.pkl")

In [11]:
def make_2d(array):
    dim1, dim2, dim3 = array.shape
    array = np.reshape(array, (dim1, dim2 * dim3))
    return array

In [12]:
epochs_2d = make_2d(epochs_array)

In [13]:
def transfer_labels2(time, epoch_labels, padding_time = 0):
  #note: time is in seconds
  shift_epochs = int(time / 5)
  padd_epochs = int(padding_time / 5)
  for index in range(len(epoch_labels)):
    #doesnt shift epoch labels if the seizure happens too early 
    if epoch_labels[index] == 1 and index - shift_epochs >= 0:
      epoch_labels[index - shift_epochs] = 1
      #padding before
      for i in range(padd_epochs):
        if (index - (shift_epochs + padd_epochs) >= 0):
          epoch_labels[index - (shift_epochs + padd_epochs)] = 1
      epoch_labels[index] = 0
  return epoch_labels

In [14]:
def get_all_epoch_labels(label_folder, time_before, padding_time):
  labels = []
  for i in range(24):
    print(i)
    if i == 0 or i == 3 or i == 12 or i == 13 or i == 17:
      continue
    if i < 10:
      f = "chb0" + str(i) + "_labels.pkl"
    else:
      f = "chb" + str(i) + "_labels.pkl"
    
    file_labels = load_epochs(f"{label_folder}/{f}") 
    file_labels = transfer_labels2(time_before, file_labels, padding_time)
    labels.append(file_labels)
    #flattening the labels array
  labels = flatten(labels)
  return labels


In [15]:
labels = get_all_epoch_labels("label_files", (80 * 60), 40 * 60)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [16]:
save_epochs(labels, "label_files/80min_transfer_40minpadd.pkl")

In [17]:
labels = np.asarray(labels)

In [18]:
def lstm(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2, dim3 = x_train.shape
  model = keras.Sequential()
  
  model.add(LSTM(128, return_sequences = True, activation = "tanh", input_shape = (dim2, dim3)))
  #latest addition: adding dropout layers
  #model.add(Dropout(0.2))
  model.add(LSTM(16, activation = 'tanh', input_shape = (128,)))
  #model.add(Dropout(0.2))
  model.add(Dense(22, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)
  

In [19]:
lstm(epochs_array[:200000], labels[:200000], epochs_array[200000:230000], labels[200000:230000], 30)

Epoch 1/30
6250/6250 [==============================] - 58s 8ms/step - loss: 0.6544 - binary_accuracy: 0.6398 - precision: 0.3148 - recall: 2.3607e-04 - false_positives: 37.0000 - false_negatives: 71995.0000 - true_positives: 17.0000 - true_negatives: 127951.0000 - val_loss: 0.6164 - val_binary_accuracy: 0.6998 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - val_false_negatives: 9005.0000 - val_true_positives: 0.0000e+00 - val_true_negatives: 20995.0000
Epoch 2/30
6250/6250 [==============================] - 45s 7ms/step - loss: 0.6414 - binary_accuracy: 0.6533 - precision: 0.6160 - recall: 0.0988 - false_positives: 4437.0000 - false_negatives: 64895.0000 - true_positives: 7117.0000 - true_negatives: 123551.0000 - val_loss: 0.6029 - val_binary_accuracy: 0.7001 - val_precision: 0.5143 - val_recall: 0.0160 - val_false_positives: 136.0000 - val_false_negatives: 8861.0000 - val_true_positives: 144.0000 - val_true_negatives: 20859.0000
Epoch 3/30
625

In [20]:
def knn(x_train, y_train, x_test, y_test):
  model = sklearn.neighbors.KNeighborsClassifier()
  model.fit(x_train, y_train)
  y_preds = model.predict(x_test)
  score = sklearn.metrics.accuracy_score(y_test, y_preds)
  return score

In [21]:
score = knn(epochs_2d[:200000], labels[:200000], epochs_2d[200000:230000], labels[200000:230000])

In [22]:
print(score)

0.6914333333333333


In [23]:
def logistic_regression(x_train, y_train, x_test, y_test):
  model = sklearn.linear_model.LogisticRegression()
  model.fit(x_train, y_train)
  y_preds = model.predict(x_test)
  score = sklearn.metrics.accuracy_score(y_test, y_preds)
  return score

In [24]:
score = logistic_regression(epochs_2d[:200000], labels[:200000], epochs_2d[200000:230000], labels[200000:230000])

In [25]:
print(score)

0.6998333333333333


In [26]:
def rnn(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2, dim3 = x_train.shape
  model = keras.Sequential()
  
  model.add(SimpleRNN(128, return_sequences = True, activation = "tanh", input_shape = (dim2, dim3)))
  #latest addition: adding dropout layers
  #model.add(Dropout(0.2))
  model.add(SimpleRNN(16, activation = 'tanh', input_shape = (128,)))
  #model.add(Dropout(0.2))
  model.add(Dense(22, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)
  

In [ ]:
rnn(epochs_array[:200000], labels[:200000], epochs_array[200000:230000], labels[200000:230000], 30)

Epoch 1/30
6250/6250 [==============================] - 102s 16ms/step - loss: 0.6373 - binary_accuracy: 0.6595 - precision_1: 0.6268 - recall_1: 0.1342 - false_positives_1: 5757.0000 - false_negatives_1: 62345.0000 - true_positives_1: 9667.0000 - true_negatives_1: 122231.0000 - val_loss: 0.6049 - val_binary_accuracy: 0.6992 - val_precision_1: 0.4832 - val_recall_1: 0.0288 - val_false_positives_1: 277.0000 - val_false_negatives_1: 8746.0000 - val_true_positives_1: 259.0000 - val_true_negatives_1: 20718.0000
Epoch 2/30
6250/6250 [==============================] - 96s 15ms/step - loss: 0.6082 - binary_accuracy: 0.6862 - precision_1: 0.6391 - recall_1: 0.2954 - false_positives_1: 12011.0000 - false_negatives_1: 50739.0000 - true_positives_1: 21273.0000 - true_negatives_1: 115977.0000 - val_loss: 0.5539 - val_binary_accuracy: 0.7504 - val_precision_1: 0.7544 - val_recall_1: 0.2498 - val_false_positives_1: 732.0000 - val_false_negatives_1: 6756.0000 - val_true_positives_1: 2249.0000 - val_t

In [ ]:
def cnn(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2, dim3 = x_train.shape
  print(dim3)
  model = keras.Sequential()
  
  model.add(Conv1D(2, 10, activation = "relu", input_shape = (dim2, dim3)))
  #latest addition: adding dropout layers
  #model.add(Dropout(0.2))
  #model.add(Conv1D(16, 50, activation = 'relu'))
  #model.add(Dropout(0.2))
  model.add(Dense(22, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)
  

In [ ]:
cnn(epochs_array[:200000], labels[:200000], epochs_array[200000:230000], labels[200000:230000], 30)

1280
Epoch 1/30
6250/6250 [==============================] - 37s 5ms/step - loss: 0.6528 - binary_accuracy: 0.6399 - precision_2: 0.3529 - recall_2: 8.3319e-05 - false_positives_2: 11.0000 - false_negatives_2: 72006.0000 - true_positives_2: 6.0000 - true_negatives_2: 127977.0000 - val_loss: 0.6230 - val_binary_accuracy: 0.6998 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - val_false_positives_2: 0.0000e+00 - val_false_negatives_2: 9005.0000 - val_true_positives_2: 0.0000e+00 - val_true_negatives_2: 20995.0000
Epoch 2/30
6250/6250 [==============================] - 28s 4ms/step - loss: 0.6404 - binary_accuracy: 0.6529 - precision_2: 0.7217 - recall_2: 0.0587 - false_positives_2: 1631.0000 - false_negatives_2: 67782.0000 - true_positives_2: 4230.0000 - true_negatives_2: 126357.0000 - val_loss: 0.6281 - val_binary_accuracy: 0.7000 - val_precision_2: 0.5625 - val_recall_2: 0.0020 - val_false_positives_2: 14.0000 - val_false_negatives_2: 8987.0000 - val_true_positives_2: 18.0000